In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functions_to_optimize import f_rastrigin
from CMA_obj import CMA_opt
from PEPG_obj import PEPG_opt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from SPSA_obj import SPSA_opt
from ADAM_opt import AdamOptimizer
from PSO_obj import PSO_opt

In [2]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import pandas as pd

#   Online training of Neural Networks

## 1) Defining a simple neural network
- Define the feed forward NN class
- Load the datasets 

In [32]:
#this is a simple single layer feed forward NN with ReLU activation and adjustable hidden layer size
class Neural_Net(nn.Module):
    def __init__(self, input_size, hidden_size, n_classes):
        
        super(Neural_Net, self).__init__()
        
        self.NN_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_classes),
            nn.Softmax(dim=1)
        )
    #forward pass
    def forward(self, X):
        
        X = nn.flatten(X)
        logits = self.NN_stack(X)
        return logits
    
    #method to reset the weights of the NN
    def reset_weights(self):
        for layer in self.NN_stack:
            if isinstance(layer,nn.Linear):
                layer.reset_parameters()

In [ ]:
#define training loop od pytorch NN as a function

def train_pytorch_NN(model, n_epochs, train_loader, test_loader, loss, optimizer, device):
    #array to store the accuracy of the model
    accuracy = []
    for epoch in range(n_epochs):
        for i, (images,labels) in enumerate(train_loader):
            #move data to gpu for faster processing
            images = images.to(device)
            labels = labels.to(device)
            #forward pass
            Y_pred = model.forward(images)
            loss_value = loss(Y_pred)
            #backward pass
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            #print accuracy every 100 steps for the test set
            if (i+1) % 100 == 0:
                correct = 0 
                total = 0
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    Y_pred = model.forward(images)
                    _, predicted = torch.max(Y_pred.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                accuracy.append = 100*correct/total
                print(f'Epoch [{epoch+1}/{n_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss_value.item()}, Test Accuracy: {accuracy}%')
    return 0

### Loading datasets
X is the input, Y the output

In [29]:
# MNIST dataset
MNIST_train = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
MNIST_test = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader_MNIST = torch.utils.data.DataLoader(dataset=MNIST_train, batch_size=100, shuffle=True)
test_loader_MNIST = torch.utils.data.DataLoader(dataset=MNIST_test, batch_size=100, shuffle=False)

X_train_MNIST, Y_train_MNIST = next(iter(train_loader_MNIST))
X_test_MNIST, Y_test_MNIST = next(iter(test_loader_MNIST))

100%|██████████| 9912422/9912422 [00:07<00:00, 1334652.90it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1198582.65it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [26]:
# Wine dataset
wine_df = pd.read_csv('winequality-red.csv')
wine_raw = wine_df.values
# split into train and test sets using pytorch randomsplit
Wine_train, Wine_test = torch.utils.data.random_split(wine_raw, [round(0.75*wine_raw.shape[0]), round(0.25*wine_raw.shape[0])])

#get the data from the pytoch object and convert to numpy
X_train_wine = np.array(Wine_train, dtype=np.float32)[:,:-1]
Y_train_wine = np.array(Wine_train, dtype=np.float32)[:,-1][np.newaxis,:].T

X_test_wine = np.array(Wine_test, dtype=np.float32)[:,:-1]
Y_test_wine = np.array(Wine_test, dtype=np.float32)[:,-1][np.newaxis,:].T

#convert to pytorch tensors for use in the NN

X_train_wine = torch.from_numpy(X_train_wine)
Y_train_wine = torch.from_numpy(Y_train_wine)

X_test_wine = torch.from_numpy(X_test_wine)
Y_test_wine = torch.from_numpy(Y_test_wine)

In [33]:
#We Now create an instance of the NN class and move if to the GPU if available
n_neurons = 100
NN_MNIST = Neural_Net(input_size=28*28, hidden_size=n_neurons, n_classes=10)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
NN_MNIST.to(device)

print(f"Using {device} device")
print(NN_MNIST)

Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
    (3): Softmax(dim=1)
  )
)


In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NN_MNIST.parameters(), lr=0.001)

n_epochs = 10

train_pytorch_NN(NN_MNIST, n_epochs, train_loader_MNIST, test_loader_MNIST, loss, optimizer, device)
        
            

In [ ]:
# Training loop for MNIST but without training the input layer (extreme learning machine)

#first reset the model we trained before
NN_MNIST.reset_weights()

#set the input layer to not require gradients
NN_MNIST.NN_stack[0].requires_grad = False

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NN_MNIST.parameters(), lr=0.001)

n_epochs = 10

train_pytorch_NN(NN_MNIST, n_epochs, train_loader_MNIST, test_loader_MNIST, loss, optimizer, device)


